In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

read_csv = pd.read_csv('dataset.csv')
oracle_statements = read_csv['Source_Statement'].astype(str).values.tolist()
postgres_statements = read_csv['Target_Statement'].astype(str).values.tolist()

# print(oracle_statements)

# Define the training data
# postgres_statements = [
#     "SELECT * FROM employees",
#     "INSERT INTO customers (name, email) VALUES ('John Doe', 'john@example.com')",
#     "UPDATE products SET price = 9.99 WHERE id = 123",
#     "DELETE FROM orders WHERE status = 'cancelled'",
# ]

# oracle_statements = [
#     "SELECT * FROM employees",
#     "INSERT INTO customers (name, email) VALUES ('John Doe', 'john@example.com')",
#     "UPDATE products SET price = 9.99 WHERE id = 123",
#     "DELETE FROM orders WHERE status = 'cancelled'",
# ]

# Create a tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(postgres_statements + oracle_statements)

# Convert text to sequences
postgres_sequences = tokenizer.texts_to_sequences(postgres_statements)
oracle_sequences = tokenizer.texts_to_sequences(oracle_statements)

# Pad sequences
max_length = max(max(len(seq) for seq in postgres_sequences), max(len(seq) for seq in oracle_sequences))
postgres_sequences = tf.keras.preprocessing.sequence.pad_sequences(postgres_sequences, maxlen=max_length)
oracle_sequences = tf.keras.preprocessing.sequence.pad_sequences(oracle_sequences, maxlen=max_length)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(oracle_statements), activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Convert oracle statements to one-hot encoded vectors
oracle_labels = tf.keras.utils.to_categorical(np.arange(len(oracle_statements)), num_classes=len(oracle_statements))

# Train the model
model.fit(postgres_sequences, oracle_labels, epochs=10, batch_size=16)



Epoch 1/10
3/3 [==============================] - 4s 49ms/step - loss: 3.5929 - accuracy: 0.0278
Epoch 2/10
3/3 [==============================] - 0s 39ms/step - loss: 3.5452 - accuracy: 0.0278
Epoch 3/10
3/3 [==============================] - 0s 37ms/step - loss: 3.5150 - accuracy: 0.0833
Epoch 4/10
3/3 [==============================] - 0s 37ms/step - loss: 3.4855 - accuracy: 0.1389
Epoch 5/10
3/3 [==============================] - 0s 42ms/step - loss: 3.4565 - accuracy: 0.2778
Epoch 6/10
3/3 [==============================] - 0s 37ms/step - loss: 3.4263 - accuracy: 0.2778
Epoch 7/10
3/3 [==============================] - 0s 40ms/step - loss: 3.3896 - accuracy: 0.3333
Epoch 8/10
3/3 [==============================] - 0s 39ms/step - loss: 3.3491 - accuracy: 0.3333
Epoch 9/10
3/3 [==============================] - 0s 39ms/step - loss: 3.3106 - accuracy: 0.3333
Epoch 10/10
3/3 [==============================] - 0s 43ms/step - loss: 3.2640 - accuracy: 0.3333


In [2]:
# Test the model
test_statement = """
begin set search_path to billing ;open out_cursor for select distinct ca.leveldetailname as location, c.customername as payer, pbd.uhid as uhid, pb.patientidentifiernumber as ipnumber, pb.patientname, awf.billno as billnumber, pb.billdate as billdate, pbdd.serviceid as servicecode, pbdd.servicename, pb.billno as newbillnumber, awf.createddate as dateofmodification, awf.oldbillingamount as originalbillamount, awf.newbillingamount as modifiedbillamount, pbdd.finaltariff as originalserviceamount, pbdd.amendmentamount as serviceamountmodified, pbdd.diffamount as amountmodified, pbd.reasonforcorrection as reasoncode, pbdd.remarks, um.firstname || ' ' || um.middlename || ' ' || um.lastname as modifiedby, um.firstname || ' ' || um.middlename || ' ' || um.lastname as initiatedby,( select listagg( um.firstname || ' ' || um.middlename || ' ' || um.lastname, ',' )within group( order by bapwf.billingapwftransid asc )from billing.ageingvalueapprovalwftrans bapwf inner join billing.ageingvalueapprovalworkflow awf on awf.billingapprovalwfid = bapwf.billingapprovalwfid inner join billing.usermaster um on um.employeeid = bapwf.wipby where awf.billno = pbd.billno and awf.flowname = 'Billing Correction' )
as approvedby, awf.workflowid from billing.ageingvalueapprovalworkflow awf inner join billing.ageingvalueapprovalwftrans bapwf on awf.billingapprovalwfid = bapwf.billingapprovalwfid inner join billing.patientbill_draft pbd on pbd.billno = awf.billno inner join billing.patientbill pb on pb.patientidentifiernumber = pbd.patientidentifierno inner join billing.patientbilldetails_draft pbdd on pbdd.billno = awf.billno inner join crm.customers c on c.customerid = pbd.payerid inner join billing.usermaster um on um.employeeid = bapwf.createdby inner join ehis.coa_struct_details ca on ca.chartid = awf.locationid where date_trunc( 'day', awf.createddate )between iv_fromdate and iv_todate and( iv_locationid is null or awf.locationid = iv_locationid )and( iv_payertypeid is null or c.customerid = iv_payertypeid )and( iv_intiatedby is null or awf.createdby = iv_intiatedby )and( iv_approvedby is null or bapwf.wipby = iv_approvedby )and awf.flowname = 'Billing Correction' and awf.status = 1 and pbdd.amendmentamount is not null order by dateofmodification asc ;
"""
test_sequence = tokenizer.texts_to_sequences([test_statement])
test_sequence = tf.keras.preprocessing.sequence.pad_sequences(test_sequence, maxlen=max_length)
predictions = model.predict(test_sequence)
predicted_index = np.argmax(predictions)
predicted_statement = oracle_statements[predicted_index]

print(f"Predicted Oracle statement: {predicted_statement}")

1/1 [==============================] - 0s 347ms/step
Predicted Oracle statement: OPEN OP_CURSOR FOR
 SELECT DISTINCT CA.LEVELDETAILNAME as Location,
 AWF.Billno as BillNumber,
 PB.Billdate as Billdate,
 NVL(C.Customername, 'NA') as Payer,
 BC.REGISTRATIONNO as UHID,
 PB.Patientidentifiernumber as IPNUMBER,
 PB.Patientname,
 ROUND(PB.Totalbillamount) as BillAmount,
 AWF.OLDBILLINGAMOUNT as CancelledAmount,
 AWF.COMPLETEDON as DateofCancellation,
 UM.FIRSTNAME || ' ' || UM.MIDDLENAME || ' ' ||
 UM.LASTNAME as Cancelledby,
 NVL(Lov.Lovdetaildescription, Lov.Lovdetailvalue) as Reasoncode,
 BC.Remarks,
 NB.BILLNO as NEWBILLNUMBER,
 NB.BILLDATE as DateofNewbill,
 (SELECT LISTAGG(UM.FIRSTNAME || ' ' || UM.MIDDLENAME || ' ' ||
 UM.LASTNAME,
 ',') WITHIN
 GROUP(
 order by BAPWF.Billingapwftransid asc)
 from BILLING.AGEINGVALUEAPPROVALWFTRANS bapwf
 inner join BILLING.AGEINGVALUEAPPROVALWORKFLOW AWF on AWF.BILLINGAPPROVALWFID =
 BAPWF.BILLINGAPPROVALWFID
 inner join billing.usermaster um on um.E